In [1]:
from google.colab import drive
drive.mount('/content/drive')

! pip install kaggle
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/Colab_Notebooks/kaggle.json ~/.kaggle/kaggle.json
! kaggle datasets download -d arashnic/learn-time-series-forecasting-from-gold-price
! unzip /content/learn-time-series-forecasting-from-gold-price.zip && rm /content/learn-time-series-forecasting-from-gold-prices.zip

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  0% 0.00/48.6k [00:00<?, ?B/s]
100% 48.6k/48.6k [00:00<00:00, 59.6MB/s]
Archive:  /content/learn-time-series-forecasting-from-gold-price.zip
  inflating: gold_price_data.csv     
rm: cannot remove '/content/learn-time-series-forecasting-from-gold-prices.zip': No such file or directory


In [2]:
import pandas as pd

df = pd.read_csv('gold_price_data.csv')
df.head()

,Date,Value
0,1970-01-01,35.2
1,1970-04-01,35.1
2,1970-07-01,35.4
3,1970-10-01,36.2
4,1971-01-01,37.4


In [3]:
import numpy as np
import pandas as pd
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from keras.preprocessing.sequence import TimeseriesGenerator

In [36]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
df = df.set_index(['Date'])
df.head()

,Value
Date,
1970-01-01,35.2
1970-04-01,35.1
1970-07-01,35.4
1970-10-01,36.2
1971-01-01,37.4


In [38]:
X = df.drop(labels=['Value'], axis=1)
y = df['Value']

In [46]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits=3)

for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [48]:
print('X_train: ', X_train.shape, 'X_test: ', X_test.shape)
print('y_train: ', y_train.shape, 'y_test: ',y_test.shape)
print(X_train.index)
print(X_test.index)
print(y_train.index)
print(y_test.index)

X_train:  (8091, 0) X_test:  (2696, 0)
y_train:  (8091,) y_test:  (2696,)
DatetimeIndex(['1970-01-01', '1970-04-01', '1970-07-01', '1970-10-01',
               '1971-01-01', '1971-04-01', '1971-07-01', '1971-10-01',
               '1972-01-03', '1972-04-03',
               ...
               '2009-10-30', '2009-11-02', '2009-11-03', '2009-11-04',
               '2009-11-05', '2009-11-06', '2009-11-09', '2009-11-10',
               '2009-11-11', '2009-11-12'],
              dtype='datetime64[ns]', name='Date', length=8091, freq=None)
DatetimeIndex(['2009-11-13', '2009-11-16', '2009-11-17', '2009-11-18',
               '2009-11-19', '2009-11-20', '2009-11-23', '2009-11-24',
               '2009-11-25', '2009-11-26',
               ...
               '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
               '2020-03-06', '2020-03-09', '2020-03-10', '2020-03-11',
               '2020-03-12', '2020-03-13'],
              dtype='datetime64[ns]', name='Date', length=2696, freq=No

In [56]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)

Empty DataFrame
Columns: []
Index: [1970-01-01 00:00:00, 1970-04-01 00:00:00, 1970-07-01 00:00:00, 1970-10-01 00:00:00, 1971-01-01 00:00:00, 1971-04-01 00:00:00, 1971-07-01 00:00:00, 1971-10-01 00:00:00, 1972-01-03 00:00:00, 1972-04-03 00:00:00, 1972-07-03 00:00:00, 1972-10-02 00:00:00, 1973-01-01 00:00:00, 1973-04-02 00:00:00, 1973-07-02 00:00:00, 1973-10-01 00:00:00, 1974-01-01 00:00:00, 1974-04-01 00:00:00, 1974-07-01 00:00:00, 1974-10-01 00:00:00, 1975-01-01 00:00:00, 1975-04-01 00:00:00, 1975-07-01 00:00:00, 1975-10-01 00:00:00, 1976-01-01 00:00:00, 1976-04-01 00:00:00, 1976-07-01 00:00:00, 1976-10-01 00:00:00, 1977-01-03 00:00:00, 1977-04-01 00:00:00, 1977-07-01 00:00:00, 1977-10-03 00:00:00, 1978-01-02 00:00:00, 1978-04-03 00:00:00, 1978-07-03 00:00:00, 1978-10-02 00:00:00, 1978-12-29 00:00:00, 1979-01-01 00:00:00, 1979-01-02 00:00:00, 1979-01-03 00:00:00, 1979-01-04 00:00:00, 1979-01-05 00:00:00, 1979-01-08 00:00:00, 1979-01-09 00:00:00, 1979-01-10 00:00:00, 1979-01-11 00:00:00

In [59]:
y_train.head()

Date
1970-01-01    35.2
1970-04-01    35.1
1970-07-01    35.4
1970-10-01    36.2
1971-01-01    37.4
Name: Value, dtype: float64

In [57]:
from sklearn.preprocessing import MinMaxScaler

y = y_train[]

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(y)
print(scaled_data.shape)
print(scaled_data)
print(len(scaled_data))


ValueError: ignored

AttributeError: ignored

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  series = tf.expand_dims(series, axis=-1)
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(window_size + 1))
  ds = ds.shuffle(shuffle_buffer)
  ds = ds.map(lambda w: (w[:-1], w[-1:]))
  return ds.batch(batch_size).prefetch(1)

train_set = windowed_dataset(scaled_data, window_size=60, batch_size=100, shuffle_buffer=1000)

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.LSTM(60),

    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),

    tf.keras.layers.Dense(1),
])

In [6]:
optimizer = tf.keras.optimizers.SGD(lr=1.0000e-04, momentum=0.9)

model.compile(loss = tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=['mae'])

history = model.fit(train_set, epochs=100)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


108/108 [==============================] - 11s 66ms/step - loss: 655.3711 - mae: 655.8711
Epoch 2/100
108/108 [==============================] - 7s 65ms/step - loss: 654.1111 - mae: 654.6111
Epoch 3/100
108/108 [==============================] - 8s 73ms/step - loss: 649.5869 - mae: 650.0869
Epoch 4/100
108/108 [==============================] - 8s 74ms/step - loss: 582.4427 - mae: 582.9427
Epoch 5/100
108/108 [==============================] - 8s 73ms/step - loss: 182.8240 - mae: 183.3224
Epoch 6/100
108/108 [==============================] - 7s 64ms/step - loss: 212.4044 - mae: 212.9034
Epoch 7/100
108/108 [==============================] - 7s 64ms/step - loss: 233.1774 - mae: 233.6766
Epoch 8/100
108/108 [==============================] - 7s 66ms/step - loss: 243.5485 - mae: 244.0476
Epoch 9/100
108/108 [==============================] - 8s 73ms/step - loss: 245.4132 - mae: 245.9123
Epoch 10/100
108/108 [==============================] - 7s 64ms/step - loss: 248.8077 - mae: 249.3065


KeyboardInterrupt: ignored